# From model to production

In [ ]:
!pip install -Uqq duckduckgo_search

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for {term}")
    return L(ddg_images(term, max_results=max_images)).itemgot("image")

In [ ]:
urls = search_images(term="grizzly bear", max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
from fastai.vision.all import *

download_url(url=urls[0], dest="data/grizzly.jpg", show_progress=False)

im = Image.open("data/grizzly.jpg")
im.to_thumb(256, 256)

In [ ]:
download_url(url=search_images(term="grizzly bear", max_images=1)[0], dest="data/grizzly.jpg", show_progress=False)
im = Image.open("data/grizzly.jpg")
im.to_thumb(256, 256)

In [ ]:
import os
from time import sleep

directory_path = "grizly_or_not"
search_terms = ["grizzly bear"]
path = Path("grizzly_or_not")

if os.path.exists(directory_path) and any(os.scandir(directory_path)):
    print("Dataset already downloaded!")
else:
    print("Downloading dataset...")
    for search_term in search_terms:
        dest = (path/search_term)
        dest.mkdir(exist_ok=True, parents=True)
        download_images(dest=dest, urls=search_images(f"{search_term} photo"))
        sleep(10)
        download_images(dest=dest, urls=search_images(f"{search_term} sun photo"))
        sleep(10)
        download_images(dest=dest, urls=search_images(f"{search_term} shade photo"))
        sleep(10)
        resize_images(path/search_term, max_size=400, dest=path/search_term)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method="squish")]
).dataloaders(path, bs=32)

In [ ]:
dls.show_batch(max_n=8)

In [ ]:
learner = vision_learner(dls, resnet34, metrics=error_rate)
learner.fine_tune(4)

In [ ]:
is_grizzly, _, probs = learner.predict(PILImage.create("data/grizzly.jpg"))
print(f"This is a {is_grizzly}!")
print(f"Probability this is a grizzly bear is: {probs[0]:.4f}")

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learner)
cleaner

In [ ]:
learn.export()

In [ ]:
learner_inf = load_learner(path/'export.pkl')

In [ ]:
learner_inf.predict('data/grizzly.jpg')

In [ ]:
learn_inf.dls.vocab

In [ ]:
import ipywidgets as widgets
uploader = widgets.FileUpload()
uploader

In [ ]:
img = PILImage.create(uploader.data[0])
is_grizzly,_,probs = learner.predict(img)
print(f"Is this a dog?: {is_dog}.")
print(f"Probability it's a grizzly bear: {probs[1].item():.6f}")

## Deploying our model on the web

<a href="https://www.tanishq.ai/blog/gradio_hf_spaces_tutorial">Gradio + HuggingFace Spaces: A Tutorial</a>